In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_excel("Online Retail.xlsx")

In [3]:
df = df.rename(columns={'Description':'ProductName'})

Dropping rows with null CustomerID

In [4]:
df.dropna(subset="CustomerID", inplace=True)

In [5]:
df = df[(df['Quantity'] > -25000) & (df['Quantity'] < 25000)]
df

,InvoiceNo,StockCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406825 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406825 non-null  object        
 1   StockCode    406825 non-null  object        
 2   ProductName  406825 non-null  object        
 3   Quantity     406825 non-null  int64         
 4   InvoiceDate  406825 non-null  datetime64[ns]
 5   UnitPrice    406825 non-null  float64       
 6   CustomerID   406825 non-null  float64       
 7   Country      406825 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [7]:
df[["Quantity", "UnitPrice"]].describe()

,Quantity,UnitPrice
count,406825.000000,406825.000000
mean,12.061422,3.460490
std,50.208125,69.315502
min,-9360.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,12540.000000,38970.000000


In [8]:
df["TotalValue"] = df["Quantity"]*df["UnitPrice"]

In [9]:
df[["Quantity", "UnitPrice", "TotalValue"]].describe()

,Quantity,UnitPrice,TotalValue
count,406825.000000,406825.000000,406825.000000
mean,12.061422,3.460490,20.402054
std,50.208125,69.315502,118.405700
min,-9360.000000,0.000000,-38970.000000
25%,2.000000,1.250000,4.200000
50%,5.000000,1.950000,11.100000
75%,12.000000,3.750000,19.500000
max,12540.000000,38970.000000,38970.000000


In [10]:
df["InvoiceNo"] = df["InvoiceNo"].astype(str)

In [11]:
df['Cancelled'] = df['InvoiceNo'].str.startswith('C').astype(int)

Creating customer table to implement customer churn and retention analysis

In [12]:
customers = df.groupby("CustomerID").agg(
    OrderCount=('InvoiceNo', 'count'),   
    TotalValue=('TotalValue', 'sum'),      
    FirstOrderDate=('InvoiceDate', 'first'),  
    LastOrderDate=('InvoiceDate', 'last'),    
    AvgQty=('Quantity', 'mean'),      
    CancelledOrders=('Cancelled', 'sum'),  
    Country=('Country', 'first')          
).reset_index()

In [13]:
customers['IsWholesaler'] = np.where(customers['AvgQty'] > 5, 1, 0)

In [14]:
customers['AvgValue'] = customers["TotalValue"]/customers['OrderCount']

In [15]:
customers['Duration'] = (customers['LastOrderDate'].dt.to_period('M') - customers['FirstOrderDate'].dt.to_period('M')).apply(lambda x: x.n)


In [16]:
customers["ValidOrders"] = customers['OrderCount']-customers['CancelledOrders']

In [17]:
customers["IsRepeat"] = np.where((customers["ValidOrders"] < 10) & (customers["ValidOrders"]>1), 1, 0)

In [18]:
customers["IsLoyal"] = np.where(customers["ValidOrders"]> 10, 1, 0)

In [19]:
customers.sort_values(by="TotalValue", ascending=False).loc[0]

CustomerID                     12347.0
OrderCount                         182
TotalValue                      4310.0
FirstOrderDate     2010-12-07 14:57:00
LastOrderDate      2011-12-07 15:52:00
AvgQty                       13.505495
CancelledOrders                      0
Country                        Iceland
IsWholesaler                         1
AvgValue                     23.681319
Duration                            12
ValidOrders                        182
IsRepeat                             0
IsLoyal                              1
Name: 0, dtype: object

In [20]:
customers[['IsWholesaler','IsRepeat', 'IsLoyal']].value_counts()

IsWholesaler  IsRepeat  IsLoyal
1             0         1          2817
0             0         1           861
1             1         0           439
              0         0           124
0             1         0            72
              0         0            58
Name: count, dtype: int64

In [21]:
customers['IsWholesaler'].value_counts()

IsWholesaler
1    3380
0     991
Name: count, dtype: int64

In [22]:
customers.to_csv("customers.csv", header=True, index=False)

In [23]:
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

In [24]:
products = df.groupby(['StockCode', 'ProductName']).agg(
    Count=('StockCode', 'size'),  # Count of product instances
    LatestMonth=('YearMonth', 'last')  # Count of unique purchasers
).reset_index()

In [25]:
fixed_date = pd.to_datetime("2011-12", format="%Y-%m")
products["UnsoldDuration"] = 12*(fixed_date.year - products["LatestMonth"].dt.year) + (fixed_date.month - products["LatestMonth"].dt.month)

In [26]:
products_time = df.groupby(['StockCode', 'ProductName', 'YearMonth']).agg(
    Count=('StockCode', 'size')
).reset_index()

In [27]:
products_time

,StockCode,ProductName,YearMonth,Count
0,10002,INFLATABLE POLITICAL GLOBE,2010-12,16
1,10002,INFLATABLE POLITICAL GLOBE,2011-01,18
2,10002,INFLATABLE POLITICAL GLOBE,2011-02,5
3,10002,INFLATABLE POLITICAL GLOBE,2011-03,5
4,10002,INFLATABLE POLITICAL GLOBE,2011-04,5
...,...,...,...,...
30964,POST,POSTAGE,2011-08,96
30965,POST,POSTAGE,2011-09,119
30966,POST,POSTAGE,2011-10,124
30967,POST,POSTAGE,2011-11,184


In [28]:
products_time['YearMonth'] = products_time['YearMonth'].dt.strftime('%Y-%m')

In [29]:
products.to_csv("products.csv", header=True, index=False)

In [30]:
products_time.to_csv("product_sales.csv", header=True, index=False)